In [1]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Using cached visions-0.7.4-py3-none-any.whl (102 kB)
Processing c:\users\kven\appdata\local\pip\cache\wheels\70\e1\52\5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655\htmlmin-0.1.12-py3-none-any.whl
  Using cached missingno-0.5.0-py3-none-any.whl (8.8 kB)
  Using cached phik-0.12.0-cp37-cp37m-win_amd64.whl (660 kB)
  Using cached tangled_up_in_unicode-0.2.0-py3-none-any.whl (4.7 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached multimethod-1.6-py3-none-any.whl (9.4 kB)
Processing c:\users\kven\appdata\local\pip\cache\wheels\4c\d5\59\5e3e297533ddb09407769762985d134135064c6831e29a914e\imagehash-4.2.1-py2.py3-none-any.whl
  Using cached charset_normalizer-2.0.7-py3-none-any.whl (38 kB)
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py

ERROR: conda 4.10.3 requires ruamel_yaml_conda>=0.11.14, which is not installed.


In [2]:
pip install petl

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for petl: filename=petl-1.7.4-py3-none-any.whl size=217003 sha256=7691e0e61626d7ccde976cc0bb8d021212116b22c52a438650640f2f20390225
  Stored in directory: c:\users\kven\appdata\local\pip\cache\wheels\75\23\fb\a0e8e350a8718579b0cde55163df77b94de835d7d116388fa5
Successfully built petl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas_profiling import ProfileReport
#from sodapy import Socrata
import pyarrow
import json
import requests
import os
import sys
import argparse
import petl as etl
from datetime import datetime

# Data Extraction #

In [2]:
def extract_CSV(csv_location:str):
    print("-----\n extracting data from CSV")
    try:
        df = pd.read_csv(csv_location)
        if len(df) > 0:
            print(f"{len(df)} rows extracted \n-----")
            return df
        else:
            print(f"{csv_location} has 0 rows of data")
    except:
            print(f"{csv_location} extraction failed")

# Data Profiling #

In [3]:
def sample_data_profiling(df,df_name:str):
    # Sample 1.000.000 rows
    sample = df.sample(1000000)

    profile = ProfileReport(sample, minimal=True)
    profile.to_file(f"{df_name}.html")
    profile.to_widgets()

In [4]:
def detailed_data_profiling(df, df_name:str):
    print('='*10)
    print(f"{df_name} -- data profiling:")
    print(f"number of rows {df.shape[0]}")
    print(f"number of columns {df.shape[1]}")
    print('-'*5)
    print(f"column names: {df.columns}")
    print('-'*5)
    print("column data types:")
    for i in df.columns:
        print(f"{i} -- dtype: {df[i].dtypes}")
    print('-'*5)
    print("Columns with null values:")
    print(df.isnull().sum())
    print('-'*5)
    print("the percentage of nulls in every column:")
    print(df.isna().sum()/(len(df))*100)
    print('-'*5)
    print("number of duplicate rows:")
    print(df.duplicated().sum())
    print('-'*5)
    print("numeric column statistics:")
    print(df.describe())
    print('='*10)

# Data Cleaning

In [5]:
def drop_unused_columns(df,to_drop:list):
    dropped_df = df.drop(to_drop, axis=1)
    return dropped_df

In [6]:
def perform_data_cleansing(df,
                               df_name:str,
                               null_values_approach = "drop",
                               duplicates_approach = "keep_first",
                               outliers_approach = 3,
                               outlier_columns = None):
        print('='*10)
        print(f"cleaning {df_name} \n-----")
        
        # remove or replace Null values
        print(f"{df_name} Null values cleaning method: {null_values_approach}")
        if null_values_approach == "drop":
            print(f"dropping {len(df[df.isna().any(axis = 1)])} Null rows")
            df = df.dropna()
            
        elif null_values_approach == "threshold_2":
            df = df.dropna(thresh = 2)
            
        elif null_values_approach == "fill_0":
            df = df.fillna(0)
            
        elif null_values_approach == "ffill":
            df = df.fillna(method = 'ffill')
        
        # handle duplicates
        print(f"-----\n{df_name} duplicate values cleaning method: {duplicates_approach}")
        if duplicates_approach == "keep_first":
            print(f"dropping {df.duplicated().sum()} duplicate rows")
            df.drop_duplicates(keep = "first")
            
        elif duplicates_approach == "keep_last":          
            df.drop_duplicates(keep = "last")
        
        # handle outliers
        if type(outliers_approach) == int and outlier_columns is not None:
            print(f"{df_name} outliers cleaning method: abs {outliers_approach}")
            from scipy import stats
            for c in outlier_columns:
                if df[c].dtypes == "int64" or df[c].dtypes == "float64":
                    df = df[(np.abs(stats.zscore(df[c])) < outliers_approach)]
        print('='*10)
        # return cleaned dataframe
        return df

# Create Dimension Tables

In [7]:
 def create_dimension(df,
                         dimension_columns:list, # [first_attribute, second, third]
                         surrogate_key_name:str,
                         surrogate_key_integer_start:int):
        
        print(f"-----\n creating dimension: {dimension_columns[0]}_dim")
        # copy full dataframe to create dimension from subset
        dim = df.copy()
        dim = dim[dimension_columns]
        
        # drop unneeded rows in hierarchy
        dim = dim.drop_duplicates(subset=[dimension_columns[0]], keep = "first")
        
        # add surrogate key
        dim.insert(0, surrogate_key_name, range(surrogate_key_integer_start,
                                                surrogate_key_integer_start+len(dim)))
        
        print(f"dimension {dimension_columns[0]}_dim created with {len(dim)} rows \n-----")
        # return the dimension as a dataframe
        return dim

# Create Fact Tables

In [8]:
def create_fact(df,
                    dimensions:list,
                    date_column = None):
        
        print(f"-----\n creating fact table")
        # copy full dataframe to create fact table
        fact = df.copy()
        
        # for every dimension, add the FK to the Fact table and remove hierarchy
        for d in dimensions:
                
            fact = fact.merge(d[[d.columns[0], d.columns[1]]],
                              left_on = d.columns[1],
                              right_on = d.columns[1],
                              how = 'left')
            
            for c in range(1, len(d.columns)):
                fact = fact.drop(d.columns[c], 1)
        
        # create date_id column in fact table to match date_dim        
        if date_column != None:
            
            import datetime as dt
            fact["date_id"] = df[date_column].dt.strftime("%m%d%Y")
            
            if date_column != "date_id":
                
                fact = fact.drop(date_column, 1)
        
        # return fact table as a dataframe        
        print(f"fact table created with {len(fact)} rows \n-----")
        return fact

# Create Bigquerry Client

In [9]:
def establish_bigquery_connection(key_path:str,
                                      scopes_url = r'https://www.googleapis.com/auth/cloud-platform'):
        # created bigquery credentials from service account key json file
        credentials = service_account.Credentials.from_service_account_file(key_path)
        
        # initiate the bigquery client with the credentials
        client = bigquery.Client(credentials=credentials, project=credentials.project_id)
        
        # return the bigquery client
        return client

# Load Tables to Bigquery

In [10]:
def load_table_to_bigquery(bq_client,
                               table,
                               dataset_name:str,
                               table_name:str):
        
        print(f"-----\n loading {table_name} to BigQuery dataset {dataset_name}")
        # define bigquery client
        client = bq_client
        
        # define location you will upload table to in bigquery
        table_ref = client.dataset(dataset_name).table(table_name)
        
        # configure load job settings
        job_config = bigquery.LoadJobConfig()
        job_config.autodetect = True
        job_config.source_format = bigquery.SourceFormat.CSV
        job_config.write_disposition = "WRITE_TRUNCATE"
        
        # initiate load job
        load_job = client.load_table_from_dataframe(table, table_ref,
                                                    job_config=job_config)
        load_job.result()
        
        # Make a BigQuery API request to check if new table was loaded successfully
        validate_table = client.get_table(table_ref)  # Make an API request.
        print(f"Loaded {validate_table.num_rows} rows and {len(validate_table.schema)} columns to {table_ref} \n-----")

# Extract & Analyze Person Data

In [11]:
person_df = extract_CSV('Motor_Vehicle_Collisions_-_Person.csv')

-----
 extracting data from CSV
4486018 rows extracted 
-----


In [13]:
#sample_data_profiling(person_df, 'Person Data')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
detailed_data_profiling(person_df, 'Person Data')

Person Data -- data profiling:
number of rows 4486018
number of columns 21
-----
column names: Index(['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'PERSON_ID',
       'PERSON_TYPE', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION',
       'EMOTIONAL_STATUS', 'BODILY_INJURY', 'POSITION_IN_VEHICLE',
       'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'COMPLAINT',
       'PED_ROLE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2',
       'PERSON_SEX'],
      dtype='object')
-----
column data types:
UNIQUE_ID -- dtype: int64
COLLISION_ID -- dtype: int64
CRASH_DATE -- dtype: object
CRASH_TIME -- dtype: object
PERSON_ID -- dtype: object
PERSON_TYPE -- dtype: object
PERSON_INJURY -- dtype: object
VEHICLE_ID -- dtype: float64
PERSON_AGE -- dtype: float64
EJECTION -- dtype: object
EMOTIONAL_STATUS -- dtype: object
BODILY_INJURY -- dtype: object
POSITION_IN_VEHICLE -- dtype: object
SAFETY_EQUIPMENT -- dtype: object
PED_LOCATION -- dtype: object
PED_ACTION -- dtype: objec

# Cleaning Person Data

In [12]:
pure_person_df = drop_unused_columns(person_df, ['UNIQUE_ID','CRASH_TIME','VEHICLE_ID','EJECTION','EMOTIONAL_STATUS','SAFETY_EQUIPMENT','PED_LOCATION','PED_ACTION','COMPLAINT','CONTRIBUTING_FACTOR_1','CONTRIBUTING_FACTOR_2'])

In [13]:
clean_person_df = perform_data_cleansing(pure_person_df,
                               'Clean Person Data',
                               null_values_approach = "drop",
                               duplicates_approach = "keep_first",
                               outliers_approach = 3,
                               outlier_columns = None)

cleaning Clean Person Data 
-----
Clean Person Data Null values cleaning method: drop
dropping 2218480 Null rows
-----
Clean Person Data duplicate values cleaning method: keep_first
dropping 0 duplicate rows


# Create DimPerson Table

In [14]:
dim_person = create_dimension(clean_person_df,
                         ['PERSON_ID','PERSON_AGE','PERSON_SEX','PERSON_TYPE','PERSON_INJURY','BODILY_INJURY','POSITION_IN_VEHICLE','PED_ROLE','COLLISION_ID'],
                         'ID',
                         1000)

-----
 creating dimension: PERSON_ID_dim
dimension PERSON_ID_dim created with 2267538 rows 
-----


In [15]:
dim_person

,ID,PERSON_ID,PERSON_AGE,PERSON_SEX,PERSON_TYPE,PERSON_INJURY,BODILY_INJURY,POSITION_IN_VEHICLE,PED_ROLE,COLLISION_ID
1,1000,4629e500-a73e-48dc-b8fb-53124d124b80,33.0,F,Occupant,Unspecified,Does Not Apply,"Front passenger, if two or more persons, inclu...",Passenger,4230587
4,1001,e038e18f-40fb-4471-99cf-345eae36e064,7.0,F,Occupant,Unspecified,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Passenger,4231168
5,1002,84bcb3a7-d201-4c61-9e30-fe29268c1074,27.0,M,Occupant,Injured,Back,Driver,Driver,4230743
8,1003,c3fc715e-203f-462d-9e8b-6a41fc378703,36.0,M,Occupant,Unspecified,Does Not Apply,Driver,Driver,4229808
10,1004,48a63e6b-b696-4a33-935c-05b4c6b70867,30.0,M,Occupant,Unspecified,Does Not Apply,Driver,Driver,4230737
...,...,...,...,...,...,...,...,...,...,...
4486005,2268533,9b86c87c-c6bb-4945-8f8c-baf204ce9341,24.0,F,Occupant,Unspecified,Does Not Apply,"Front passenger, if two or more persons, inclu...",Passenger,4473965
4486006,2268534,587974d6-da4b-4cbf-977b-6712743a9049,38.0,F,Occupant,Unspecified,Does Not Apply,Driver,Driver,4473513
4486013,2268535,644e0d22-6d5a-4d03-8c12-69f0d0f31f60,22.0,M,Occupant,Injured,Entire Body,Driver,Driver,4473812
4486014,2268536,f3368a0f-20e0-42ba-988c-f61f6218568e,43.0,M,Occupant,Injured,Neck,Driver,Driver,4473578


# Extract & Analyze Collision Data

In [16]:
collision_df = extract_CSV('Motor_Vehicle_Collisions_-_Crashes.csv')

-----
 extracting data from CSV


C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


1837353 rows extracted 
-----


In [20]:
#sample_data_profiling(collision_df, 'Collision Data')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
#detailed_data_profiling(collision_df, 'Collision Data')

Collision Data -- data profiling:
number of rows 1837353
number of columns 29
-----
column names: Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')
-----
column data types:
CRASH DATE -- dtype: object
CRASH TIME -- dtype: object
BOROUGH -- dtype: object
ZIP CODE -- dtyp

# Cleaning Collision Data

In [17]:
collision_df = collision_df.rename(columns={"CRASH DATE":"CRASH_DATE","CRASH TIME":"CRASH_TIME"})

In [18]:
pure_collision_df = drop_unused_columns(collision_df, ['OFF STREET NAME','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5','VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4','VEHICLE TYPE CODE 5'])

In [19]:
clean_collision_df = perform_data_cleansing(pure_collision_df,
                               'Clean Collision Data',
                               null_values_approach = "drop",
                               duplicates_approach = "keep_first",
                               outliers_approach = 3,
                               outlier_columns = None)

cleaning Clean Collision Data 
-----
Clean Collision Data Null values cleaning method: drop
dropping 1008997 Null rows
-----
Clean Collision Data duplicate values cleaning method: keep_first
dropping 0 duplicate rows


In [20]:
#population information retrieved from data.cityofnewyork.us based on New York City Population By Neighborhood Tabulation Areas
population = []
for value in clean_collision_df['BOROUGH']:
    if value == 'BROOKLYN':
        population.append(4970026)
    elif value == 'BRONX':
        population.append(2717758)
    elif value == 'MANHATTAN':
        population.append(3123068)
    elif value == 'QUEENS':
        population.append(4460101)
    elif value == 'STATEN ISLAND':
        population.append(912458)
        
clean_collision_df['POPULATION'] = population

C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [21]:
clean_collision_df

,CRASH_DATE,CRASH_TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,NUMBER OF PERSONS INJURED,...,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,POPULATION
13,05/21/2019,22:50,BROOKLYN,11201.0,40.697540,-73.983120,"(40.69754, -73.98312)",GOLD STREET,CONCORD STREET,0.0,...,0,0,0,0,Passing or Lane Usage Improper,Unspecified,4136992,�MBU,Taxi,4970026
17,03/31/2021,22:20,BROOKLYN,11234.0,40.626457,-73.918000,"(40.626457, -73.918)",RALPH AVENUE,AVENUE K,1.0,...,0,0,1,0,Driver Inexperience,Unspecified,4403773,Sedan,Sedan,4970026
18,04/06/2021,22:58,STATEN ISLAND,10312.0,40.526894,-74.167280,"(40.526894, -74.16728)",BARCLAY AVENUE,HYLAN BOULEVARD,7.0,...,0,0,7,0,Failure to Yield Right-of-Way,Unsafe Speed,4405244,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,912458
25,04/16/2021,17:40,BRONX,10474.0,40.815000,-73.894020,"(40.815, -73.89402)",GARRISON AVENUE,LONGWOOD AVENUE,0.0,...,0,0,0,0,Unspecified,Unspecified,4407900,Sedan,Sedan,2717758
28,04/16/2021,16:35,BRONX,10475.0,40.890076,-73.819855,"(40.890076, -73.819855)",BOSTON ROAD,ROPES AVENUE,2.0,...,0,0,2,0,Unsafe Speed,Unspecified,4408143,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,2717758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837347,07/07/2012,17:35,BROOKLYN,11235.0,40.575802,-73.961137,"(40.5758019, -73.9611371)",BRIGHTON 6 STREET,BRIGHTWATER COURT,0.0,...,0,0,0,0,Unspecified,Unspecified,113584,PASSENGER VEHICLE,PASSENGER VEHICLE,4970026
1837348,07/06/2012,15:09,MANHATTAN,10035.0,40.801235,-73.941815,"(40.8012354, -73.9418153)",EAST 119 STREET,PARK AVENUE,0.0,...,0,0,0,0,Unspecified,Unspecified,59654,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,3123068
1837349,07/03/2012,17:30,QUEENS,11102.0,40.774711,-73.933386,"(40.7747112, -73.9333863)",27 AVENUE,4 STREET,2.0,...,0,0,2,0,Failure to Yield Right-of-Way,Unspecified,272592,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,4460101
1837350,07/01/2012,15:30,BROOKLYN,11236.0,40.645032,-73.919978,"(40.6450318, -73.9199775)",RALPH AVENUE,CLARENDON ROAD,0.0,...,0,0,0,0,Unspecified,Unspecified,135041,SMALL COM VEH(4 TIRES),PASSENGER VEHICLE,4970026


# Create DimCollision Table

In [22]:
dim_collision = create_dimension(clean_collision_df,
                         ['COLLISION_ID'],
                         'ID',
                         1)

-----
 creating dimension: COLLISION_ID_dim
dimension COLLISION_ID_dim created with 828356 rows 
-----


In [23]:
dim_collision

,ID,COLLISION_ID
13,1,4136992
17,2,4403773
18,3,4405244
25,4,4407900
28,5,4408143
...,...,...
1837347,828352,113584
1837348,828353,59654
1837349,828354,272592
1837350,828355,135041


# Create DimLocation Table

In [24]:
dim_location = create_dimension(clean_collision_df,
                         ['LOCATION','LATITUDE','LONGITUDE','ZIP CODE','BOROUGH','POPULATION','COLLISION_ID'],
                         'LOCATION_ID',
                         10000)

-----
 creating dimension: LOCATION_dim
dimension LOCATION_dim created with 68986 rows 
-----


In [25]:
dim_location['ZIP CODE'] = dim_location['ZIP CODE'].astype(str)

In [26]:
dim_location = dim_location.rename(columns={"ZIP CODE":"ZIP_CODE"})

In [27]:
dim_location

,LOCATION_ID,LOCATION,LATITUDE,LONGITUDE,ZIP_CODE,BOROUGH,POPULATION,COLLISION_ID
13,10000,"(40.69754, -73.98312)",40.697540,-73.983120,11201.0,BROOKLYN,4970026,4136992
17,10001,"(40.626457, -73.918)",40.626457,-73.918000,11234.0,BROOKLYN,4970026,4403773
18,10002,"(40.526894, -74.16728)",40.526894,-74.167280,10312.0,STATEN ISLAND,912458,4405244
25,10003,"(40.815, -73.89402)",40.815000,-73.894020,10474.0,BRONX,2717758,4407900
28,10004,"(40.890076, -73.819855)",40.890076,-73.819855,10475.0,BRONX,2717758,4408143
...,...,...,...,...,...,...,...,...
1836390,78981,"(40.8138322, -73.8936336)",40.813832,-73.893634,10474.0,BRONX,2717758,76226
1836522,78982,"(40.5854402, -74.1484498)",40.585440,-74.148450,10314.0,STATEN ISLAND,912458,293244
1836523,78983,"(40.558305, -74.1645302)",40.558305,-74.164530,10312.0,STATEN ISLAND,912458,293191
1836922,78984,"(40.5882381, -73.9222706)",40.588238,-73.922271,11229.0,BROOKLYN,4970026,116333


# Extract & Analyze Vehicle Data

In [28]:
vehicle_df = extract_CSV('Motor_Vehicle_Collisions_-_Vehicles.csv')

-----
 extracting data from CSV


C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


3683400 rows extracted 
-----


In [33]:
#sample_data_profiling(vehicle_df, 'Vehicle Data')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
#detailed_data_profiling(vehicle_df, 'Vehicle Data')

Vehicle Data -- data profiling:
number of rows 3683400
number of columns 25
-----
column names: Index(['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'VEHICLE_ID',
       'STATE_REGISTRATION', 'VEHICLE_TYPE', 'VEHICLE_MAKE', 'VEHICLE_MODEL',
       'VEHICLE_YEAR', 'TRAVEL_DIRECTION', 'VEHICLE_OCCUPANTS', 'DRIVER_SEX',
       'DRIVER_LICENSE_STATUS', 'DRIVER_LICENSE_JURISDICTION', 'PRE_CRASH',
       'POINT_OF_IMPACT', 'VEHICLE_DAMAGE', 'VEHICLE_DAMAGE_1',
       'VEHICLE_DAMAGE_2', 'VEHICLE_DAMAGE_3', 'PUBLIC_PROPERTY_DAMAGE',
       'PUBLIC_PROPERTY_DAMAGE_TYPE', 'CONTRIBUTING_FACTOR_1',
       'CONTRIBUTING_FACTOR_2'],
      dtype='object')
-----
column data types:
UNIQUE_ID -- dtype: int64
COLLISION_ID -- dtype: int64
CRASH_DATE -- dtype: object
CRASH_TIME -- dtype: object
VEHICLE_ID -- dtype: object
STATE_REGISTRATION -- dtype: object
VEHICLE_TYPE -- dtype: object
VEHICLE_MAKE -- dtype: object
VEHICLE_MODEL -- dtype: object
VEHICLE_YEAR -- dtype: float64
TRAVEL_DIRECTION 

# Cleaning Vehicle Data

In [31]:
pure_vehicle_df = drop_unused_columns(vehicle_df,['UNIQUE_ID','CRASH_DATE','CRASH_TIME','STATE_REGISTRATION','VEHICLE_MODEL','VEHICLE_YEAR','TRAVEL_DIRECTION','DRIVER_SEX','DRIVER_LICENSE_STATUS','DRIVER_LICENSE_JURISDICTION','PRE_CRASH','VEHICLE_DAMAGE_1','VEHICLE_DAMAGE_2','VEHICLE_DAMAGE_3','PUBLIC_PROPERTY_DAMAGE','PUBLIC_PROPERTY_DAMAGE_TYPE','CONTRIBUTING_FACTOR_2'])

In [32]:
pure_vehicle_df

,COLLISION_ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_OCCUPANTS,POINT_OF_IMPACT,VEHICLE_DAMAGE,CONTRIBUTING_FACTOR_1
0,100201,1,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,Unspecified
1,4213082,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,1.0,Left Front Bumper,Left Front Quarter Panel,Driver Inattention/Distraction
2,3307608,2,TAXI,NaN,NaN,NaN,NaN,Driver Inattention/Distraction
3,3308693,1,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,Unspecified
4,297666,1,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,Other Vehicular
...,...,...,...,...,...,...,...,...
3683395,4473887,91295ba4-8ce9-49fc-86e4-942c1c4bfef2,Bus,NaN,NaN,NaN,NaN,Passing Too Closely
3683396,4473169,50e6b948-35d0-4140-b49d-449891e05130,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,1.0,Center Back End,Center Back End,Unspecified
3683397,4473377,02da5e75-e9c6-41b5-a469-d421b4dcc134,Station Wagon/Sport Utility Vehicle,JEEP -CAR/SUV,0.0,Left Rear Quarter Panel,Left Rear Quarter Panel,Unspecified
3683398,4473596,d05e0cdf-69e6-4803-afe4-df7f12d12521,Station Wagon/Sport Utility Vehicle,MAZD -CAR/SUV,1.0,Left Front Bumper,Left Front Bumper,Driver Inexperience


In [33]:
clean_vehicle_df = perform_data_cleansing(pure_vehicle_df,
                               'Clean Collision Data',
                               null_values_approach = "drop",
                               duplicates_approach = "keep_first",
                               outliers_approach = 3,
                               outlier_columns = None)

cleaning Clean Collision Data 
-----
Clean Collision Data Null values cleaning method: drop
dropping 1827075 Null rows
-----
Clean Collision Data duplicate values cleaning method: keep_first
dropping 0 duplicate rows


In [34]:
clean_vehicle_df

,COLLISION_ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_OCCUPANTS,POINT_OF_IMPACT,VEHICLE_DAMAGE,CONTRIBUTING_FACTOR_1
1,4213082,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,1.0,Left Front Bumper,Left Front Quarter Panel,Driver Inattention/Distraction
5,3434155,219456,4 dr sedan,MERZ -CAR/SUV,2.0,Right Front Bumper,Right Front Bumper,Driver Inattention/Distraction
6,4229067,c53b43d9-419a-4ab1-9361-3f2979078d89,Bus,FRHT-TRUCK/BUS,13.0,Left Front Quarter Panel,Left Front Quarter Panel,Unspecified
7,3503027,672828,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,2.0,Center Front End,Center Front End,Driver Inattention/Distraction
10,3487936,554272,Convertible,VOLK -CAR/SUV,1.0,Right Rear Bumper,Right Rear Bumper,Unspecified
...,...,...,...,...,...,...,...,...
3683394,4473863,40435077-2168-4b89-a6ee-803c81e4cef6,Sedan,CHRY -CAR/SUV,1.0,Right Front Bumper,Right Front Bumper,Unspecified
3683396,4473169,50e6b948-35d0-4140-b49d-449891e05130,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,1.0,Center Back End,Center Back End,Unspecified
3683397,4473377,02da5e75-e9c6-41b5-a469-d421b4dcc134,Station Wagon/Sport Utility Vehicle,JEEP -CAR/SUV,0.0,Left Rear Quarter Panel,Left Rear Quarter Panel,Unspecified
3683398,4473596,d05e0cdf-69e6-4803-afe4-df7f12d12521,Station Wagon/Sport Utility Vehicle,MAZD -CAR/SUV,1.0,Left Front Bumper,Left Front Bumper,Driver Inexperience


# Create DimVehicle Table

In [35]:
dim_vehicle = create_dimension(clean_vehicle_df,
                         ['VEHICLE_ID','VEHICLE_TYPE','VEHICLE_MAKE','COLLISION_ID'],
                         'ID',
                         7000)

-----
 creating dimension: VEHICLE_ID_dim
dimension VEHICLE_ID_dim created with 1856325 rows 
-----


In [36]:
dim_vehicle

,ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,COLLISION_ID
1,7000,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,4213082
5,7001,219456,4 dr sedan,MERZ -CAR/SUV,3434155
6,7002,c53b43d9-419a-4ab1-9361-3f2979078d89,Bus,FRHT-TRUCK/BUS,4229067
7,7003,672828,Station Wagon/Sport Utility Vehicle,FORD -CAR/SUV,3503027
10,7004,554272,Convertible,VOLK -CAR/SUV,3487936
...,...,...,...,...,...
3683394,1863320,40435077-2168-4b89-a6ee-803c81e4cef6,Sedan,CHRY -CAR/SUV,4473863
3683396,1863321,50e6b948-35d0-4140-b49d-449891e05130,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,4473169
3683397,1863322,02da5e75-e9c6-41b5-a469-d421b4dcc134,Station Wagon/Sport Utility Vehicle,JEEP -CAR/SUV,4473377
3683398,1863323,d05e0cdf-69e6-4803-afe4-df7f12d12521,Station Wagon/Sport Utility Vehicle,MAZD -CAR/SUV,4473596


# Create DimFactors Table

In [37]:
dim_factors = create_dimension(clean_vehicle_df,
                         ['COLLISION_ID','CONTRIBUTING_FACTOR_1'],
                         'FACTOR_ID',
                         15000)

-----
 creating dimension: COLLISION_ID_dim
dimension COLLISION_ID_dim created with 1037795 rows 
-----


In [38]:
dim_factors

,FACTOR_ID,COLLISION_ID,CONTRIBUTING_FACTOR_1
1,15000,4213082,Driver Inattention/Distraction
5,15001,3434155,Driver Inattention/Distraction
6,15002,4229067,Unspecified
7,15003,3503027,Driver Inattention/Distraction
10,15004,3487936,Unspecified
...,...,...,...
3683379,1052790,4473303,Turning Improperly
3683381,1052791,4473157,Unspecified
3683390,1052792,4473333,Unspecified
3683392,1052793,4473912,Unspecified


# Create DimDate Table

In [39]:
dim_date = clean_collision_df[['CRASH_DATE','CRASH_TIME','COLLISION_ID']]

In [40]:
dim_date['CRASH_DATE'] = pd.to_datetime(dim_date['CRASH_DATE'], format='%m/%d/%Y')

C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
dim_date['HOURLY_INTERVAL'] = pd.DatetimeIndex(dim_date['CRASH_TIME']).hour
dim_date['DAY'] = pd.DatetimeIndex(dim_date['CRASH_DATE']).day
dim_date['WEEK'] = pd.DatetimeIndex(dim_date['CRASH_DATE']).week
dim_date['MONTH'] = pd.DatetimeIndex(dim_date['CRASH_DATE']).month
dim_date['YEAR'] = pd.DatetimeIndex(dim_date['CRASH_DATE']).year

C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kven\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior o

In [43]:
dim_date

,CRASH_DATE,CRASH_TIME,COLLISION_ID,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR
13,2019-05-21,22:50,4136992,22,21,21,5,2019
17,2021-03-31,22:20,4403773,22,31,13,3,2021
18,2021-04-06,22:58,4405244,22,6,14,4,2021
25,2021-04-16,17:40,4407900,17,16,15,4,2021
28,2021-04-16,16:35,4408143,16,16,15,4,2021
...,...,...,...,...,...,...,...,...
1837347,2012-07-07,17:35,113584,17,7,27,7,2012
1837348,2012-07-06,15:09,59654,15,6,27,7,2012
1837349,2012-07-03,17:30,272592,17,3,27,7,2012
1837350,2012-07-01,15:30,135041,15,1,26,7,2012


# Create Person Facts Table

In [44]:
dim_person.head(3)

,ID,PERSON_ID,PERSON_AGE,PERSON_SEX,PERSON_TYPE,PERSON_INJURY,BODILY_INJURY,POSITION_IN_VEHICLE,PED_ROLE,COLLISION_ID
1,1000,4629e500-a73e-48dc-b8fb-53124d124b80,33.0,F,Occupant,Unspecified,Does Not Apply,"Front passenger, if two or more persons, inclu...",Passenger,4230587
4,1001,e038e18f-40fb-4471-99cf-345eae36e064,7.0,F,Occupant,Unspecified,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Passenger,4231168
5,1002,84bcb3a7-d201-4c61-9e30-fe29268c1074,27.0,M,Occupant,Injured,Back,Driver,Driver,4230743


In [45]:
dim_collision.head(3)

,ID,COLLISION_ID
13,1,4136992
17,2,4403773
18,3,4405244


In [46]:
dim_vehicle.head(3)

,ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,COLLISION_ID
1,7000,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,4213082
5,7001,219456,4 dr sedan,MERZ -CAR/SUV,3434155
6,7002,c53b43d9-419a-4ab1-9361-3f2979078d89,Bus,FRHT-TRUCK/BUS,4229067


In [47]:
dim_date.head(3)

,CRASH_DATE,CRASH_TIME,COLLISION_ID,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR
13,2019-05-21,22:50,4136992,22,21,21,5,2019
17,2021-03-31,22:20,4403773,22,31,13,3,2021
18,2021-04-06,22:58,4405244,22,6,14,4,2021


In [48]:
fact_person = clean_person_df.copy()

In [49]:
fact_person_merged_vehicle = fact_person.merge(dim_vehicle,on=['COLLISION_ID'],how='inner')

In [50]:
fact_person_merged_vehicle_merged_date = pd.concat([dim_date,fact_person_merged_vehicle])

In [51]:
fact_person_merged_vehicle_merged_date_merged_person = fact_person_merged_vehicle_merged_date.merge(dim_person,on=['PERSON_ID'],how='inner')

In [52]:
fact_person_merged_vehicle_merged_date_merged_person.drop_duplicates(subset ="PERSON_ID",
                     keep = 'first', inplace = True)

In [53]:
fact_person_merged_vehicle_merged_date_merged_person

,CRASH_DATE,CRASH_TIME,COLLISION_ID_x,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR,PERSON_ID,PERSON_TYPE_x,...,VEHICLE_MAKE,ID_y,PERSON_AGE_y,PERSON_SEX_y,PERSON_TYPE_y,PERSON_INJURY_y,BODILY_INJURY_y,POSITION_IN_VEHICLE_y,PED_ROLE_y,COLLISION_ID_y
0,10/25/2019,NaN,4230587,NaN,NaN,NaN,NaN,NaN,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,...,NISS -CAR/SUV,1000,33.0,F,Occupant,Unspecified,Does Not Apply,"Front passenger, if two or more persons, inclu...",Passenger,4230587
2,10/25/2019,NaN,4230587,NaN,NaN,NaN,NaN,NaN,cd27406c-5cc1-4a3c-9636-3e0385b0b307,Occupant,...,NISS -CAR/SUV,1654,6.0,M,Occupant,Unspecified,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Passenger,4230587
4,10/25/2019,NaN,4230587,NaN,NaN,NaN,NaN,NaN,47e9fb58-1df7-46cc-a2ff-9c2deb0f3e54,Occupant,...,NISS -CAR/SUV,1790,33.0,M,Occupant,Unspecified,Does Not Apply,Driver,Driver,4230587
6,10/25/2019,NaN,4230587,NaN,NaN,NaN,NaN,NaN,86cd7a22-51be-47ed-9e2c-9f3278c043e7,Occupant,...,NISS -CAR/SUV,2198,82.0,M,Occupant,Unspecified,Does Not Apply,Driver,Driver,4230587
8,10/25/2019,NaN,4231168,NaN,NaN,NaN,NaN,NaN,e038e18f-40fb-4471-99cf-345eae36e064,Occupant,...,TOYT -CAR/SUV,1001,7.0,F,Occupant,Unspecified,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Passenger,4231168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4444236,10/14/2021,NaN,4467017,NaN,NaN,NaN,NaN,NaN,78693e36-e331-4e8e-bd42-609130760f3c,Occupant,...,AUDI -CAR/SUV,2268250,31.0,M,Occupant,Unspecified,Does Not Apply,Driver,Driver,4467017
4444237,10/29/2021,NaN,4472318,NaN,NaN,NaN,NaN,NaN,1b95c74c-1fb9-45c8-af31-7d43f6b49124,Occupant,...,NISS -CAR/SUV,2268374,51.0,F,Occupant,Unspecified,Does Not Apply,Driver,Driver,4472318
4444239,10/29/2021,NaN,4472318,NaN,NaN,NaN,NaN,NaN,fa7e2da5-4dbd-4cc2-bcf7-9a0729ab89aa,Occupant,...,NISS -CAR/SUV,2268510,42.0,M,Occupant,Injured,Entire Body,"Front passenger, if two or more persons, inclu...",Passenger,4472318
4444241,10/08/2021,NaN,4465408,NaN,NaN,NaN,NaN,NaN,b996eae8-5520-48a8-ae7e-7b0dca688922,Occupant,...,HOND -CAR/SUV,2268455,30.0,F,Occupant,Injured,Neck,Driver,Driver,4465408


In [54]:
person_facts = drop_unused_columns(fact_person_merged_vehicle_merged_date_merged_person,['PERSON_TYPE_x','PERSON_INJURY_x','PERSON_AGE_x','BODILY_INJURY_x','POSITION_IN_VEHICLE_x','PED_ROLE_x','PERSON_SEX_x','COLLISION_ID_y','ID_x','VEHICLE_TYPE','VEHICLE_MAKE','CRASH_TIME','HOURLY_INTERVAL','DAY','WEEK','MONTH','YEAR','ID_y','PERSON_AGE_y','PERSON_SEX_y','PERSON_TYPE_y','PERSON_INJURY_y','BODILY_INJURY_y','POSITION_IN_VEHICLE_y','PED_ROLE_y'])

In [55]:
person_facts.drop_duplicates(keep='first')

,CRASH_DATE,COLLISION_ID_x,PERSON_ID,VEHICLE_ID
0,10/25/2019,4230587,4629e500-a73e-48dc-b8fb-53124d124b80,37080180-400c-431e-90eb-c09d736fd95e
2,10/25/2019,4230587,cd27406c-5cc1-4a3c-9636-3e0385b0b307,37080180-400c-431e-90eb-c09d736fd95e
4,10/25/2019,4230587,47e9fb58-1df7-46cc-a2ff-9c2deb0f3e54,37080180-400c-431e-90eb-c09d736fd95e
6,10/25/2019,4230587,86cd7a22-51be-47ed-9e2c-9f3278c043e7,37080180-400c-431e-90eb-c09d736fd95e
8,10/25/2019,4231168,e038e18f-40fb-4471-99cf-345eae36e064,280b021d-1f42-4267-9e96-48808086a209
...,...,...,...,...
4444236,10/14/2021,4467017,78693e36-e331-4e8e-bd42-609130760f3c,d11b7f43-0f87-4495-b47f-ede64a30bf4d
4444237,10/29/2021,4472318,1b95c74c-1fb9-45c8-af31-7d43f6b49124,a49519d1-5f83-4dd9-ad8b-e3afda7b1897
4444239,10/29/2021,4472318,fa7e2da5-4dbd-4cc2-bcf7-9a0729ab89aa,a49519d1-5f83-4dd9-ad8b-e3afda7b1897
4444241,10/08/2021,4465408,b996eae8-5520-48a8-ae7e-7b0dca688922,0bd5e373-90ad-470f-8f85-fd1d42b1f955


# Create Crash Facts Table

In [56]:
dim_collision.head(3)

,ID,COLLISION_ID
13,1,4136992
17,2,4403773
18,3,4405244


In [57]:
dim_vehicle.head(3)

,ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,COLLISION_ID
1,7000,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,4213082
5,7001,219456,4 dr sedan,MERZ -CAR/SUV,3434155
6,7002,c53b43d9-419a-4ab1-9361-3f2979078d89,Bus,FRHT-TRUCK/BUS,4229067


In [58]:
dim_date.head(3)

,CRASH_DATE,CRASH_TIME,COLLISION_ID,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR
13,2019-05-21,22:50,4136992,22,21,21,5,2019
17,2021-03-31,22:20,4403773,22,31,13,3,2021
18,2021-04-06,22:58,4405244,22,6,14,4,2021


In [59]:
dim_location.head(3)

,LOCATION_ID,LOCATION,LATITUDE,LONGITUDE,ZIP_CODE,BOROUGH,POPULATION,COLLISION_ID
13,10000,"(40.69754, -73.98312)",40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
17,10001,"(40.626457, -73.918)",40.626457,-73.91800,11234.0,BROOKLYN,4970026,4403773
18,10002,"(40.526894, -74.16728)",40.526894,-74.16728,10312.0,STATEN ISLAND,912458,4405244


In [60]:
fact_crash = clean_collision_df.copy()

In [61]:
fact_crash_merged_vehicle = fact_crash.merge(dim_vehicle, on=['COLLISION_ID'],how='inner')

In [62]:
fact_crash_merged_vehicle_merged_date = pd.concat([dim_date,fact_crash_merged_vehicle])

In [63]:
fact_crash_merged_vehicle_merged_date_merged_location = fact_crash_merged_vehicle_merged_date.merge(dim_location,on=['LOCATION'],how='inner')

In [64]:
fact_crash_merged_vehicle_merged_date_merged_location

,CRASH_DATE,CRASH_TIME,COLLISION_ID_x,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR,BOROUGH_x,ZIP CODE,...,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,LOCATION_ID,LATITUDE_y,LONGITUDE_y,ZIP_CODE,BOROUGH_y,POPULATION_y,COLLISION_ID_y
0,05/21/2019,22:50,4136992,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11201.0,...,985541e4-3517-4547-824e-4360ecff37a6,AMBU,AMBULANCE,10000,40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
1,05/21/2019,22:50,4136992,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11201.0,...,3b813680-8669-40ac-ad2f-5df148412212,Taxi,TOYT -CAR/SUV,10000,40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
2,01/14/2021,17:10,4384098,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11201.0,...,90722d72-d97f-44ce-9317-9b3205963769,Sedan,MERZ -CAR/SUV,10000,40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
3,01/14/2021,17:10,4384098,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11201.0,...,e8ea9784-a209-41ad-8634-aebc5306eb77,Ambulance,FORD-TRUCK/BUS,10000,40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
4,10/16/2021,0:36,4467540,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11201.0,...,778de9ba-bc6d-4de0-90de-6617042aff76,Taxi,NISS -CAR/SUV,10000,40.697540,-73.98312,11201.0,BROOKLYN,4970026,4136992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635131,12/15/2015,10:45,3580632,NaN,NaN,NaN,NaN,NaN,STATEN ISLAND,10304.0,...,1206574,Sedan,NISS -CAR/SUV,57633,40.846700,-73.91576,10304.0,STATEN ISLAND,912458,3580632
635132,12/18/2014,9:15,3812222,NaN,NaN,NaN,NaN,NaN,QUEENS,11365,...,765c5b9c-3cfa-4d09-998f-3b22f378cf3b,Garbage or Refuse,mack,72745,40.730526,-73.80144,11365,QUEENS,4460101,3812222
635133,12/18/2014,9:15,3812222,NaN,NaN,NaN,NaN,NaN,QUEENS,11365,...,c0a8ed45-2778-4d58-80a6-53de7e2b01b2,Sedan,MERZ -CAR/SUV,72745,40.730526,-73.80144,11365,QUEENS,4460101,3812222
635134,08/14/2014,11:30,3501088,NaN,NaN,NaN,NaN,NaN,BROOKLYN,11218,...,661979,Sedan,BMW -CAR/SUV,74144,40.642498,-73.96846,11218,BROOKLYN,4970026,3501088


In [65]:
crash_facts = drop_unused_columns(fact_crash_merged_vehicle_merged_date_merged_location, ['BOROUGH_x', 'LATITUDE_x',
       'LONGITUDE_x', 'ON STREET NAME', 'CROSS STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2'
       ,'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'ID', 'VEHICLE_TYPE', 'VEHICLE_MAKE',
       'HOURLY_INTERVAL', 'DAY', 'WEEK',
       'MONTH', 'YEAR', 'LATITUDE_y', 'LONGITUDE_y',
       'BOROUGH_y', 'POPULATION_y', 'COLLISION_ID_y','ZIP CODE'])

In [66]:
crash_facts

,CRASH_DATE,CRASH_TIME,COLLISION_ID_x,LOCATION,POPULATION_x,VEHICLE_ID,LOCATION_ID,ZIP_CODE
0,05/21/2019,22:50,4136992,"(40.69754, -73.98312)",4970026.0,985541e4-3517-4547-824e-4360ecff37a6,10000,11201.0
1,05/21/2019,22:50,4136992,"(40.69754, -73.98312)",4970026.0,3b813680-8669-40ac-ad2f-5df148412212,10000,11201.0
2,01/14/2021,17:10,4384098,"(40.69754, -73.98312)",4970026.0,90722d72-d97f-44ce-9317-9b3205963769,10000,11201.0
3,01/14/2021,17:10,4384098,"(40.69754, -73.98312)",4970026.0,e8ea9784-a209-41ad-8634-aebc5306eb77,10000,11201.0
4,10/16/2021,0:36,4467540,"(40.69754, -73.98312)",4970026.0,778de9ba-bc6d-4de0-90de-6617042aff76,10000,11201.0
...,...,...,...,...,...,...,...,...
635131,12/15/2015,10:45,3580632,"(40.8467, -73.91576)",912458.0,1206574,57633,10304.0
635132,12/18/2014,9:15,3812222,"(40.730526, -73.80144)",4460101.0,765c5b9c-3cfa-4d09-998f-3b22f378cf3b,72745,11365
635133,12/18/2014,9:15,3812222,"(40.730526, -73.80144)",4460101.0,c0a8ed45-2778-4d58-80a6-53de7e2b01b2,72745,11365
635134,08/14/2014,11:30,3501088,"(40.642498, -73.96846)",4970026.0,661979,74144,11218


In [67]:
crash_facts.drop_duplicates(keep='first')

,CRASH_DATE,CRASH_TIME,COLLISION_ID_x,LOCATION,POPULATION_x,VEHICLE_ID,LOCATION_ID,ZIP_CODE
0,05/21/2019,22:50,4136992,"(40.69754, -73.98312)",4970026.0,985541e4-3517-4547-824e-4360ecff37a6,10000,11201.0
1,05/21/2019,22:50,4136992,"(40.69754, -73.98312)",4970026.0,3b813680-8669-40ac-ad2f-5df148412212,10000,11201.0
2,01/14/2021,17:10,4384098,"(40.69754, -73.98312)",4970026.0,90722d72-d97f-44ce-9317-9b3205963769,10000,11201.0
3,01/14/2021,17:10,4384098,"(40.69754, -73.98312)",4970026.0,e8ea9784-a209-41ad-8634-aebc5306eb77,10000,11201.0
4,10/16/2021,0:36,4467540,"(40.69754, -73.98312)",4970026.0,778de9ba-bc6d-4de0-90de-6617042aff76,10000,11201.0
...,...,...,...,...,...,...,...,...
635131,12/15/2015,10:45,3580632,"(40.8467, -73.91576)",912458.0,1206574,57633,10304.0
635132,12/18/2014,9:15,3812222,"(40.730526, -73.80144)",4460101.0,765c5b9c-3cfa-4d09-998f-3b22f378cf3b,72745,11365
635133,12/18/2014,9:15,3812222,"(40.730526, -73.80144)",4460101.0,c0a8ed45-2778-4d58-80a6-53de7e2b01b2,72745,11365
635134,08/14/2014,11:30,3501088,"(40.642498, -73.96846)",4970026.0,661979,74144,11218


# Create Vehicle Facts Table

In [68]:
dim_vehicle.head(3)

,ID,VEHICLE_ID,VEHICLE_TYPE,VEHICLE_MAKE,COLLISION_ID
1,7000,0553ab4d-9500-4cba-8d98-f4d7f89d5856,Station Wagon/Sport Utility Vehicle,TOYT -CAR/SUV,4213082
5,7001,219456,4 dr sedan,MERZ -CAR/SUV,3434155
6,7002,c53b43d9-419a-4ab1-9361-3f2979078d89,Bus,FRHT-TRUCK/BUS,4229067


In [69]:
dim_factors.head(3)

,FACTOR_ID,COLLISION_ID,CONTRIBUTING_FACTOR_1
1,15000,4213082,Driver Inattention/Distraction
5,15001,3434155,Driver Inattention/Distraction
6,15002,4229067,Unspecified


In [70]:
dim_collision.head(3)

,ID,COLLISION_ID
13,1,4136992
17,2,4403773
18,3,4405244


In [71]:
dim_date.head(3)

,CRASH_DATE,CRASH_TIME,COLLISION_ID,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR
13,2019-05-21,22:50,4136992,22,21,21,5,2019
17,2021-03-31,22:20,4403773,22,31,13,3,2021
18,2021-04-06,22:58,4405244,22,6,14,4,2021


In [72]:
fact_vehicle = clean_vehicle_df.copy()

In [73]:
fact_vehicle_merged_vehicle = fact_vehicle.merge(dim_vehicle,on=['COLLISION_ID'],how='inner')

In [74]:
fact_vehicle_merged_vehicle_merged_date = fact_vehicle_merged_vehicle.merge(dim_date,on=['COLLISION_ID'],how='inner')

In [75]:
fact_vehicle_merged_vehicle_merged_date_merged_factor = fact_vehicle_merged_vehicle_merged_date.merge(dim_factors,on=['COLLISION_ID'],how='inner')

In [76]:
fact_vehicle_merged_vehicle_merged_date_merged_factor

,COLLISION_ID,VEHICLE_ID_x,VEHICLE_TYPE_x,VEHICLE_MAKE_x,VEHICLE_OCCUPANTS,POINT_OF_IMPACT,VEHICLE_DAMAGE,CONTRIBUTING_FACTOR_1_x,ID,VEHICLE_ID_y,...,VEHICLE_MAKE_y,CRASH_DATE,CRASH_TIME,HOURLY_INTERVAL,DAY,WEEK,MONTH,YEAR,FACTOR_ID,CONTRIBUTING_FACTOR_1_y
0,3487936,554272,Convertible,VOLK -CAR/SUV,1.0,Right Rear Bumper,Right Rear Bumper,Unspecified,7004,554272,...,VOLK -CAR/SUV,2016-07-22,15:40,15,22,29,7,2016,15004,Unspecified
1,3487936,554272,Convertible,VOLK -CAR/SUV,1.0,Right Rear Bumper,Right Rear Bumper,Unspecified,80365,554271,...,HOND -CAR/SUV,2016-07-22,15:40,15,22,29,7,2016,15004,Unspecified
2,3487936,554271,4 dr sedan,HOND -CAR/SUV,1.0,Center Front End,Center Front End,Driver Inattention/Distraction,7004,554272,...,VOLK -CAR/SUV,2016-07-22,15:40,15,22,29,7,2016,15004,Unspecified
3,3487936,554271,4 dr sedan,HOND -CAR/SUV,1.0,Center Front End,Center Front End,Driver Inattention/Distraction,80365,554271,...,HOND -CAR/SUV,2016-07-22,15:40,15,22,29,7,2016,15004,Unspecified
4,3499697,650962,Sedan,NISS -CAR/SUV,2.0,Left Side Doors,Left Side Doors,Unspecified,7005,650962,...,NISS -CAR/SUV,2016-08-13,21:05,21,13,32,8,2016,15005,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343321,4473641,b90ba5a8-ff19-458f-8d4c-fbb9b0ca5dee,Pick-up Truck,DODG -CAR/SUV,1.0,Center Front End,Center Front End,Driver Inattention/Distraction,1863256,b90ba5a8-ff19-458f-8d4c-fbb9b0ca5dee,...,DODG -CAR/SUV,2021-11-01,17:57,17,1,44,11,2021,1052773,Driver Inattention/Distraction
1343322,4473807,f0dc0d15-b301-408a-a6e2-338645cf2875,Sedan,HOND -CAR/SUV,1.0,Right Front Bumper,Center Front End,Turning Improperly,1863263,f0dc0d15-b301-408a-a6e2-338645cf2875,...,HOND -CAR/SUV,2021-10-27,16:00,16,27,43,10,2021,1052775,Turning Improperly
1343323,4473807,f0dc0d15-b301-408a-a6e2-338645cf2875,Sedan,HOND -CAR/SUV,1.0,Right Front Bumper,Center Front End,Turning Improperly,1863311,c98f0f0c-574c-483d-abe2-646fd8a4ce58,...,Nova,2021-10-27,16:00,16,27,43,10,2021,1052775,Turning Improperly
1343324,4473807,c98f0f0c-574c-483d-abe2-646fd8a4ce58,Bus,Nova,1.0,Left Rear Bumper,Left Rear Bumper,Unspecified,1863263,f0dc0d15-b301-408a-a6e2-338645cf2875,...,HOND -CAR/SUV,2021-10-27,16:00,16,27,43,10,2021,1052775,Turning Improperly


In [77]:
vehicle_facts = drop_unused_columns(fact_vehicle_merged_vehicle_merged_date_merged_factor, ['VEHICLE_TYPE_x', 'VEHICLE_MAKE_x',
       'VEHICLE_OCCUPANTS', 'POINT_OF_IMPACT', 'VEHICLE_DAMAGE',
       'CONTRIBUTING_FACTOR_1_y', 'ID','FACTOR_ID',
       'VEHICLE_ID_y', 'VEHICLE_TYPE_y', 'VEHICLE_MAKE_y',
       'CRASH_TIME', 'HOURLY_INTERVAL', 'DAY', 'WEEK', 'MONTH', 'YEAR'])

In [78]:
vehicle_facts.drop_duplicates(keep='first')

,COLLISION_ID,VEHICLE_ID_x,CONTRIBUTING_FACTOR_1_x,CRASH_DATE
0,3487936,554272,Unspecified,2016-07-22
2,3487936,554271,Driver Inattention/Distraction,2016-07-22
4,3499697,650962,Unspecified,2016-08-13
5,4229563,86a294b4-6672-4a7e-8357-39d6d2eff9f7,Unspecified,2019-10-21
7,4229563,ca744068-6013-47f0-a661-a0fbb579eb47,Unspecified,2019-10-21
...,...,...,...,...
1343317,4473267,3e2caed9-0b40-4f70-a661-8cc7e8b353b8,Passing or Lane Usage Improper,2021-11-01
1343319,4473267,c85dfca0-4653-48a9-bf25-0c94770adf44,Unspecified,2021-11-01
1343321,4473641,b90ba5a8-ff19-458f-8d4c-fbb9b0ca5dee,Driver Inattention/Distraction,2021-11-01
1343322,4473807,f0dc0d15-b301-408a-a6e2-338645cf2875,Turning Improperly,2021-10-27


# Load Tables to Bigquery

In [131]:
key_path = r'C:\Users\Kven\Desktop\baruch\courses\CIS9440\project\Milestone_3\tidal-heading-324722-9a9aa5afb1f2.json'

In [133]:
big_query_client = establish_bigquery_connection(key_path
                                      ,scopes_url = r'https://www.googleapis.com/auth/cloud-platform')

In [135]:
# load DimPerson Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_person,
                               'milestone3',
                               'dim_person')

-----
 loading dim_person to BigQuery dataset milestone3
Loaded 2267538 rows and 10 columns to tidal-heading-324722.milestone3.dim_person 
-----


In [136]:
# load DimCollision Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_collision,
                               'milestone3',
                               'dim_collision')

-----
 loading dim_collision to BigQuery dataset milestone3
Loaded 828356 rows and 2 columns to tidal-heading-324722.milestone3.dim_collision 
-----


In [151]:
# load DimLocation Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_location,
                               'milestone3',
                               'dim_location')

-----
 loading dim_location to BigQuery dataset milestone3
Loaded 68986 rows and 8 columns to tidal-heading-324722.milestone3.dim_location 
-----


In [152]:
# load DimVehicle Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_vehicle,
                               'milestone3',
                               'dim_vehicle')

-----
 loading dim_vehicle to BigQuery dataset milestone3
Loaded 1856325 rows and 5 columns to tidal-heading-324722.milestone3.dim_vehicle 
-----


In [153]:
# load DimFactors Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_factors,
                               'milestone3',
                               'dim_factors')

-----
 loading dim_factors to BigQuery dataset milestone3
Loaded 1037795 rows and 3 columns to tidal-heading-324722.milestone3.dim_factors 
-----


In [154]:
# load DimDate Table to Bigquery
load_table_to_bigquery(big_query_client,
                               dim_date,
                               'milestone3',
                               'dim_date')

-----
 loading dim_date to BigQuery dataset milestone3
Loaded 828356 rows and 8 columns to tidal-heading-324722.milestone3.dim_date 
-----


In [168]:
# load Person Facts Table to Bigquery
load_table_to_bigquery(big_query_client,
                               person_facts,
                               'milestone3',
                               'person_facts')

-----
 loading person_facts to BigQuery dataset milestone3
Loaded 2247010 rows and 4 columns to tidal-heading-324722.milestone3.person_facts 
-----


In [169]:
# load Crash Facts Table to Bigquery
load_table_to_bigquery(big_query_client,
                               crash_facts,
                               'milestone3',
                               'crash_facts')

-----
 loading crash_facts to BigQuery dataset milestone3
Loaded 635136 rows and 8 columns to tidal-heading-324722.milestone3.crash_facts 
-----


In [170]:
# load Vehicle Facts Table to Bigquery
load_table_to_bigquery(big_query_client,
                               vehicle_facts,
                               'milestone3',
                               'vehicle_facts')

-----
 loading vehicle_facts to BigQuery dataset milestone3
Loaded 1343326 rows and 4 columns to tidal-heading-324722.milestone3.vehicle_facts 
-----
